# 3D Droplet Oscillation (Part 2, Postprocessing - Frequency change and quasi-periodicity)

Results published: hopefully at some point!

This is part of the BoSSS-long-term validation test suite, which consists of several computationally expensive test-cases (runtime in the order of days), which are performed on a regular basis in order to validate the physical correctness of BoSSS simulations.

### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

In [ ]:
int numSess = wmg.Sessions.Count();
numSess

43

In [ ]:
//add database manually:
//OpenOrCreateDatabase(@"\\dc1\userspace\smuda\Databases\OscillatingDroplet3D");
OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D");
//OpenOrCreateDatabase(@"\\130.83.248.207\ValidationTests\OscillatingDroplet3D");

## Observing sessions

In [ ]:
var workSess = wmg.Sessions.Skip(numSess);
workSess

#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart2*	08/01/2022 09:37:12	624802c2...
#1: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart1_withReInit*	07/29/2022 18:26:25	5e0fa717...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03_restart2	07/11/2022 13:39:12	95fa458f...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_restart2	07/11/2022 13:38:30	b97b80ef...
#4: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m3_Oh01_eta04*	07/11/2022 13:56:43	5480a5ac...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m4_Oh01_eta01*	07/11/2022 13:56:41	d98b11be...
#6: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m3_Oh01_eta015*	07/11/2022 13:49:48	53060394...
#7: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m4_Oh01_eta01_thirdOrderInit*	07/11/2022 13:49:03	325af26a...
#8: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr2_m3_Oh01_eta015_thirdOrderInit*	07/11/2022 13:48:12	bc10dc66...
#9: OscillatingDroplet3D	OD3D

In [ ]:
using System.IO;

# Study Overview

In [ ]:
string[] caseS = new string[] { "m2_Oh01_eta04", "m2_Oh01_eta02", "m2_Oh01_eta01" };
string[] gridS = new string[] { "J686" }; //, "J432" };
bool[] ShouldAnaInit = new bool[] { false, true };
bool[] useNewton = new bool[] { false };

In [ ]:
var studySess = new List<ISessionInfo>();
foreach(string _case in caseS) {
    foreach(bool _init in ShouldAnaInit) {
        foreach(var sess in workSess) {
            if(sess.Name.Contains(_case) && sess.Name.Contains(gridS[0])) {
                if(!(_init ^ sess.Name.Contains("_thirdOrderInit"))) {
                    studySess.Add(sess);
                    break;
                }
            }
        }
    }   
}

In [ ]:
//var studySess = workSess.Skip(3).Take(18);
studySess

#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_restart3	06/13/2022 15:31:38	ceb7f024...
#1: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart2*	08/01/2022 09:37:12	624802c2...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_restart3	06/15/2022 08:37:47	95fc59fd...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_thirdOrderInit_restart3	06/15/2022 09:32:54	d8e3f635...
#4: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_restart3	06/15/2022 08:37:41	dd97e2f1...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_thirdOrderInit_restart3	06/15/2022 08:37:38	0d35c433...


# Plotting  droplet metrics - aspect ratio L/W

In [ ]:
Plot2Ddata[,] PlotTableDM = new Plot2Ddata[caseS.Length,1];
// Plot2Ddata[,] PlotTableDM_unprocessed = new Plot2Ddata[caseS.Length,1];
for(int iCol = 0; iCol < 1; iCol++) {
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
for(int iGrd = 0; iGrd < gridS.Length; iGrd++) {   
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
//for(int iNew = 0; iNew < useNewton.Length; iNew++) {

    string _case = caseS[iRow];
    string _grd = gridS[iGrd];

    // if(_case == "case2" || _case == "case3")
    //     continue; 

    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_grd)
                                      && (sess.Name.Contains("_thirdOrderInit") == ShouldAnaInit[iAna]));
                                      //&& (sess.Name.Contains("_Newton") == useNewton[iNew]));


    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    if(SIs.IsNullOrEmpty()) 
        continue;

    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);


    Stack<ISessionInfo>  procSIs = new Stack<ISessionInfo>();
    procSIs.Push(SI);
    var currSI = SI;
    var rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    while(!rSIs.IsNullOrEmpty()) {
        var rSI = rSIs.Single();
        procSIs.Push(rSI);
        currSI = rSI;
        rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    }
    //Console.WriteLine("number of processed Sessions for {0}: {1}", _case, procSIs.Count);

    var plot = new Plot2Ddata();
    // var plot2 = new Plot2Ddata();

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.DashType = DashTypes.Solid;
    fmt.LineWidth = 2;

    if (ShouldAnaInit[iAna]) {
        fmt.LineColor= LineColors.Red;
    } else {
        fmt.LineColor= LineColors.Blue;
    }
    
 
    string name = "BoSSS-" + _grd;
    // if(iAMR > 0)
    //     name = name + "-amr";
    if(ShouldAnaInit[iAna])
        name = name + "-3OrdInit"; 
    // if(useNewton[iNew])
    //     name = name + "-Newton";


    double[] time = new double[0];
    double[] arLW = new double[0];

    double[] arL = new double[0];
    double[] arWx = new double[0];
    double[] arWy = new double[0];

    foreach(var pSI in procSIs) {

        //Console.WriteLine("processing Session: {0}", pSI.Name);
        IDictionary<string, IList<double>> tab = null;
        try {
            tab = pSI.ReadTabulatedTextFileAsDoubles("DropletMetrics.txt", '\t');
        } catch (Exception e) {
            Console.Error.WriteLine(e.Message);
            Console.WriteLine("Skipping " + pSI);
            
            // var dummyPlot = new Plot2Ddata();
            // dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
            // PlotTableDM[iRow,iCol] = dummyPlot;
            continue;
        }
        
        time = time.Concat(tab["time"]).ToArray();

        double[] L = tab["L"].ToArray(); 
        double[] Wx = tab["Wx"].ToArray(); 
        double[] Wy = tab["Wy"].ToArray();
        double[] LW = new double[L.Length];
        for (int i = 0; i < L.Length; i++) {
            LW[i] = 0.5 * ((L[i] / (2.0*Wx[i])) + (L[i] / (2.0*Wy[i])));
        }
        arLW = arLW.Concat(LW).ToArray();

        arL = arL.Concat(L).ToArray();
        arWx = arWx.Concat(Wx).ToArray();
        arWy = arWy.Concat(Wy).ToArray();

    }

    // plot2.AddDataGroup("L", time, arL, fmt);
    // plot2.AddDataGroup("Wx", time, arWx, fmt);
    // plot2.AddDataGroup("Wy", time, arWy, fmt);

    List<double> arLW_lst = new List<double>();
    List<double> time_lst = new List<double>();
    double dataThreshold = 0.01;
    for(int i = 0; i < time.Length; i++) {
        if(arLW[i] > dataThreshold) {
            arLW_lst.Add(arLW[i]);
            time_lst.Add(time[i]);
        }
    }
    arLW = arLW_lst.ToArray();
    time = time_lst.ToArray();

    plot.AddDataGroup(name, time, arLW, fmt);
   
    plot.ShowLegend = true;  

    if(PlotTableDM[iRow,iCol]==null) {
        PlotTableDM[iRow,iCol] = plot;
        // PlotTableDM_unprocessed[iRow,iCol] = plot2;
    } else {
        PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
        // PlotTableDM_unprocessed[iRow,iCol] = PlotTableDM_unprocessed[iRow,iCol].Merge(plot2);
    }

}

if(PlotTableDM[iRow,iCol] != null) {
    PlotTableDM[iRow,iCol].ShowLegend = true; 
    //PlotTableDM[iRow,iCol].LegendFont = 10;
    if(iCol == 0) {
       PlotTableDM[iRow,iCol].Title = caseS[iRow];
       PlotTableDM[iRow,iCol].Xlabel = "time t";
       PlotTableDM[iRow,iCol].Ylabel = "aspect ratio L/W";
    }
}
}
}
}

In [ ]:
PlotTableDM

index dataGroups LogX LogY LogBaseX LogBaseY LogX2 LogY2 XrangeMin XrangeMax YrangeMin YrangeMax Y2rangeMin Y2rangeMax X2rangeMin X2rangeMax Xlabel X2label Ylabel Y2label Title .. 0 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m2_Oh01_eta04 1 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m2_Oh01_eta02 2 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m2_Oh01_eta01

### Period durations

In [ ]:
Plot2Ddata[,] PlotTablePD = new Plot2Ddata[caseS.Length, 1];
for(int iRow = 0; iRow <  caseS.Length; iRow++) { // loop over cases
    Plot2Ddata plot = PlotTableDM[iRow,0];
    double[] time = plot.dataGroups[0].Abscissas;
    double[] LW = plot.dataGroups[0].Values;

    List<double> times_max = new List<double>();    // times at maxima
    List<double> times_min = new List<double>();    // times at minima
    List<double> times_oneNeg = new List<double>();    // times at neg value 1 crossings (+ -> -)
    List<double> times_onePos = new List<double>();    // times at pos value 1 crossings (- -> +)
    double value_max = LW[0];
    double value_min = double.MaxValue;
    double time_maxmin = 0;
    bool search_max = true;     // if false the local minima is searched for
    for(int i = 1; i < time.Length; i++) {
        double currVal = LW[i];
        if(search_max && currVal > value_max) {
            value_max = currVal;
            time_maxmin = time[i];
        } else if(!search_max && currVal < value_min) {
            value_min = currVal;
            time_maxmin = time[i];
        }
        double prevVal = LW[i-1];
        if(prevVal >= 1 && currVal < 1) {   // value 1 neg crossing
            times_oneNeg.Add(time[i]);
            times_max.Add(time_maxmin);
            value_max = 1.0;
            search_max = false;
        } else if(prevVal < 1 && currVal >= 1) {   // value 1 pos crossing
            times_onePos.Add(time[i]);
            times_min.Add(time_maxmin);
            value_min = 1.0;
            search_max = true;
        }
    }

    List<double> periodTimes = new List<double>();
    List<double> timesAtPstart = new List<double>(); 

    for(int i = 0; i < times_max.Count - 1; i++) {
        periodTimes.Add(times_max.ElementAt(i+1) - times_max.ElementAt(i)); 
        timesAtPstart.Add(times_max.ElementAt(i));
    }
    for(int i = 0; i < times_min.Count - 1; i++) {
        periodTimes.Add(times_min.ElementAt(i+1) - times_min.ElementAt(i)); 
        timesAtPstart.Add(times_min.ElementAt(i));
    }
    for(int i = 0; i < times_oneNeg.Count - 1; i++) {
        periodTimes.Add(times_oneNeg.ElementAt(i+1) - times_oneNeg.ElementAt(i)); 
        timesAtPstart.Add(times_oneNeg.ElementAt(i));
    }
    for(int i = 0; i < times_onePos.Count - 1; i++) {
        periodTimes.Add(times_onePos.ElementAt(i+1) - times_onePos.ElementAt(i)); 
        timesAtPstart.Add(times_onePos.ElementAt(i));
    }

    Plot2Ddata procPlot = new Plot2Ddata();
    var fmt = new PlotFormat();
    fmt.Style = Styles.Points;
    fmt.PointSize = 1;
    procPlot.AddDataGroup(plot.dataGroups[0].Name, timesAtPstart, periodTimes, fmt);

    PlotTablePD[iRow, 0] = procPlot;
}


## Case 1 (m2_Oh01_eta04)

In [ ]:
var pltTab1 = PlotTableDM[0,0];
pltTab1.Title

m2_Oh01_eta04

In [ ]:
pltTab1.XrangeMin = 0;
pltTab1.XrangeMax = 7;
var gp = pltTab1.ToGnuplot();
gp.PlotSVG()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m2 O h01 e ta04 
 
 
 BoSSS-J686 
 
 
 
 
 BoSSS-J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M466.4,93.1 L519.8,93.1 M71.9,207.1 L72.4,207.1 L72.9,207.2 L73.4,207.3 L73.8,207.5 L74.3,207.7
 L74.8,207.9 L75.3,208.2 L75.8,208.6 L76.3,209.0 L76.7,209.4 L77.2,209.8 L77.7,210.2 L78.2,210.7
 L78.7,211.2 L79.2,211.7 L79.7,212.3 L80.1,212.8 L80.6,213.4 L81.1,214.0 L81.6,214.6 L82.1,215.2
 L82.6,215.8 L83.0,216.4 L83.5,217.1 L84.0,217.7 L84.5,218.4 L85.0,219.0 L85.5,219.7 L85.9,220.4
 L86.4,221.1 L86.9,221.8 L87.4,222.5 L87.9,223.2 L88.4,223.9 L88.9,224.7 L89.3,225.4 L89.8,226.1
 L90.3,226.9 L90.8,227.6 L91.3,228.4 L91.8,229.1 L92.2,229.9 L92.7,230.6 L93.2,231.4 L93.7,232.2
 L94.2,232.9 L94.7,233.7 L95.2,234.5 L95.6,235.2 L96.1,236.0 L96.6,236.8 L97.1,237.6 L97.6,238.4
 L98.1,239.2 L98.5,240.0 L99.0,240.8 L99.5,241.6 L100.0,242.4 L100.5,243.2 L101.0,244.0 L101.5,244.9
 L101.9,245.7 L102.4,246.5 L102.9,247.3 L103.4,248.2 L103.9,249.0 L104.4,249.8 L104.8,250.7 L105.3,251.6
 L105.8,252.5 L106.3,253.3 L106.8,254.2 L107.3,255.0 L107.7,255.9 L108.2,256.7 L108.7,257.6 L109.2,258.5
 L109.7,259.4 L110.2,260.3 L110.7,261.2 L111.1,262.1 L111.6,263.0 L112.1,263.9 L112.6,264.8 L113.1,265.8
 L113.6,266.7 L114.0,267.6 L114.5,268.6 L115.0,269.6 L115.5,270.6 L116.0,271.6 L116.5,272.6 L117.0,273.7
 L117.4,274.6 L117.9,275.7 L118.4,276.7 L118.9,277.8 L119.4,278.8 L119.9,279.8 L120.3,280.9 L120.8,282.0
 L121.3,283.1 L121.8,284.2 L122.3,285.4 L122.8,286.5 L123.2,287.6 L123.7,288.8 L124.2,289.9 L124.7,291.1
 L125.2,292.2 L125.7,293.4 L126.2,294.6 L126.6,295.8 L127.1,297.0 L127.6,298.2 L128.1,299.4 L128.6,300.7
 L129.1,301.9 L129.5,303.1 L130.0,304.4 L130.5,305.7 L131.0,306.9 L131.5,308.2 L132.0,309.5 L132.5,310.8
 L132.9,312.0 L133.4,313.3 L133.9,314.6 L134.4,315.9 L134.9,317.2 L135.4,318.5 L135.8,319.8 L136.3,321.1
 L136.8,322.4 L137.3,323.7 L137.8,325.0 L138.3,326.3 L138.8,327.6 L139.2,328.9 L139.7,330.1 L140.2,331.4
 L140.7,332.7 L141.2,334.0 L141.7,335.2 L142.1,336.5 L142.6,337.8 L143.1,339.0 L143.6,340.3 L144.1,341.5
 L144.6,342.8 L145.0,344.0 L145.5,345.3 L146.0,346.5 L146.5,347.7 L147.0,348.9 L147.5,350.1 L148.0,351.3
 L148.4,352.5 L148.9,353.7 L149.4,354.9 L149.9,356.0 L150.4,357.2 L150.9,358.3 L151.3,359.4 L151.8,360.5
 L152.3,361.6 L152.8,362.7 L153.3,363.8 L153.8,364.9 L154.3,366.0 L154.7,367.0 L155.2,368.1 L155.7,369.1
 L156.2,370.2 L156.7,371.2 L157.2,372.1 L157.6,373.1 L158.1,374.1 L158.6,375.1 L159.1,376.0 L160.1,377.8
 L160.6,378.7 L161.0,379.6 L161.5,380.5 L162.0,381.3 L162.5,382.1 L163.0,382.9 L163.5,383.7 L163.9,384.5
 L164.4,385.3 L164.9,386.1 L165.4,386.9 L165.9,387.7 L166.4,388.4 L166.8,389.2 L167.3,389.9 L167.8,390.6
 L168.3,391.3 L168.8,392.0 L169.3,392.5 L169.8,393.1 L170.2,393.7 L170.7,394.2 L171.2,394.8 L171.7,395.3
 L172.2,395.8 L172.7,396.3 L173.1,396.8 L173.6,397.4 L174.1,397.8 L174.6,398.3 L175.1,398.8 L175.6,399.3
 L176.1,399.7 L176.5,400.1 L177.0,400.6 L177.5,400.9 L178.0,401.4 L178.5,401.7 L179.0,402.1 L179.4,402.5
 L179.9,402.8 L180.4,403.1 L180.9,403.5 L181.4,403.8 L181.9,404.0 L182.3,404.3 L182.8,404.6 L183.3,404.8
 L183.8,405.1 L184.3,405.3 L184.8,405.5 L185.3,405.7 L185.7,405.9 L186.2,406.1 L186.7,406.3 L187.2,406.4
 L187.7,406.6 L188.2,406.7 L188.6,406.8 L189.1,407.0 L189.6,407.1 L190.1,407.1 L190.6,407.2 L191.1,407.3
 L191.6,407.4 L192.0,407.4 L192.5,407.4 L193.0,407.5 L193.5,407.5 L19

## Case 2 (m2_Oh01_eta02)

In [ ]:
var pltTab2 = PlotTableDM[1,0];
pltTab2.Title

m2_Oh01_eta02

In [ ]:
pltTab2.XrangeMin = 0;
pltTab2.XrangeMax = 7;
var gp = pltTab2.ToGnuplot();
gp.PlotSVG()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.8 
 
 
 
 
 0.9 
 
 
 
 
 1 
 
 
 
 
 1.1 
 
 
 
 
 1.2 
 
 
 
 
 1.3 
 
 
 
 
 1.4 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m2 O h01 e ta02 
 
 
 BoSSS-J686 
 
 
 
 
 BoSSS-J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M466.4,93.1 L519.8,93.1 M71.9,122.0 L72.4,122.0 L72.9,122.1 L73.4,122.2 L73.8,122.3 L74.3,122.5
 L74.8,122.8 L75.3,123.1 L75.8,123.4 L76.3,123.8 L76.7,124.3 L77.2,124.7 L77.7,125.3 L78.2,125.8
 L78.7,126.5 L79.2,127.1 L79.7,127.8 L80.1,128.5 L80.6,129.3 L81.1,130.1 L81.6,130.9 L82.1,131.8
 L82.6,132.7 L83.0,133.6 L83.5,134.6 L84.0,135.6 L84.5,136.6 L85.0,137.7 L85.5,138.8 L85.9,139.9
 L86.4,141.1 L86.9,142.3 L87.4,143.5 L87.9,144.7 L88.4,146.0 L88.9,147.3 L89.3,148.6 L89.8,149.9
 L90.3,151.3 L90.8,152.7 L91.3,154.1 L91.8,155.6 L92.2,157.1 L92.7,158.5 L93.2,160.1 L93.7,161.6
 L94.2,163.2 L94.7,164.8 L95.2,166.4 L95.6,168.0 L96.1,169.7 L96.6,171.4 L97.1,173.1 L97.6,174.8
 L98.1,176.6 L98.5,178.4 L99.0,180.2 L99.5,182.0 L100.0,183.8 L100.5,185.7 L101.0,187.6 L101.5,189.5
 L101.9,191.4 L102.4,193.4 L102.9,195.4 L103.4,197.3 L103.9,199.4 L104.4,201.4 L104.8,203.5 L105.3,205.6
 L105.8,207.7 L106.3,209.9 L106.8,212.1 L107.3,214.2 L107.7,216.4 L108.2,218.7 L108.7,220.9 L109.2,223.2
 L109.7,225.5 L110.2,227.7 L110.7,230.0 L111.1,232.4 L111.6,234.8 L112.1,237.2 L112.6,239.6 L113.1,242.0
 L113.6,244.5 L114.0,246.9 L114.5,249.4 L115.0,252.0 L115.5,254.5 L116.0,257.0 L116.5,259.6 L117.0,262.2
 L117.4,264.8 L117.9,267.2 L118.4,269.9 L118.9,272.6 L119.4,275.1 L119.9,277.8 L120.3,280.5 L120.8,283.2
 L121.3,285.9 L121.8,288.5 L122.3,291.2 L122.8,294.0 L123.2,296.7 L123.7,299.3 L124.2,302.1 L124.7,304.9
 L125.2,307.7 L125.7,310.3 L126.2,313.1 L126.6,315.9 L127.1,318.7 L127.6,321.4 L128.1,324.2 L128.6,326.9
 L129.1,329.6 L129.5,332.4 L130.0,335.2 L130.5,338.0 L131.0,340.7 L131.5,343.5 L132.0,346.2 L132.5,349.0
 L132.9,351.7 L133.4,354.4 L133.9,357.1 L134.4,359.8 L134.9,362.5 L135.4,365.2 L135.8,367.9 L136.3,370.5
 L136.8,373.2 L137.3,375.8 L137.8,378.4 L138.3,381.0 L138.8,383.6 L139.2,386.1 L139.7,388.7 L140.2,391.2
 L140.7,393.7 L141.2,396.2 L141.7,398.7 L142.1,401.2 L142.6,403.6 L143.1,406.1 L143.6,408.5 L144.1,410.8
 L144.6,413.2 L145.0,415.6 L145.5,417.9 L146.0,420.2 L146.5,422.5 L147.0,424.7 L147.5,426.9 L148.0,429.1
 L148.4,431.4 L148.9,433.5 L149.4,435.7 L149.9,437.8 L150.4,440.0 L150.9,442.0 L151.3,444.1 L151.8,446.1
 L152.3,448.1 L152.8,450.1 L153.3,452.0 L153.8,453.9 L154.3,455.8 L154.7,457.5 L155.2,459.4 L155.7,461.2
 L156.2,463.0 L156.7,464.7 L157.2,466.4 L157.6,468.1 L158.1,469.8 L158.6,471.4 L159.1,473.0 L159.6,474.5
 L160.1,476.1 L160.6,477.6 L161.0,479.1 L161.5,480.5 L162.0,481.9 L162.5,483.3 L163.0,484.7 L163.5,486.0
 L163.9,487.3 L164.4,488.6 L164.9,489.8 L165.4,491.0 L165.9,492.2 L166.4,493.4 L171.7,504.0 L172.2,504.8
 L172.7,505.7 L173.1,506.5 L173.6,507.1 L174.1,507.9 L174.6,508.6 L175.1,509.2 L175.6,509.9 L176.1,510.4
 L176.5,511.0 L177.0,511.5 L177.5,512.0 L178.0,512.5 L178.5,512.9 L179.0,513.3 L179.4,513.7 L179.9,514.1
 L180.4,514.4 L180.9,514.7 L181.4,515.0 L181.9,515.3 L191.1,515.0 L191.6,514.8 L192.0,514.5 L192.5,514.2
 L193.0,513.8 L193.5,513.5 L194.0,513.1 L194.5,512.7 L194.9,512.2 L195.4,511.8 L195.9,511.3 L196.4,510.8
 L196.9,510.3 L197.4,509.8 L197.9,509.2 L198.3,508.6 L198.8,508.0 L199.3,507.4 L199.8,506.7 L200.3,505.9
 L200.8,505.2 L201.2,504.5 L201.7,503.8 L202.2,503.1 L202.7,502.3 L203.2,501.5 L203.7,500.7 L204.1,499.9
 L204.6,499.0 L205.1,498.1 L205.6,497.2 L

## Case 3 (m2_Oh01_eta01)

In [ ]:
var pltTab3 = PlotTableDM[2,0];
pltTab3.Title

m2_Oh01_eta01

In [ ]:
pltTab3.XrangeMin = 0;
pltTab3.XrangeMax = 7;
var gp = pltTab3.ToGnuplot();
gp.PlotSVG()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.9 
 
 
 
 
 0.95 
 
 
 
 
 1 
 
 
 
 
 1.05 
 
 
 
 
 1.1 
 
 
 
 
 1.15 
 
 
 
 
 1.2 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m2 O h01 e ta01 
 
 
 BoSSS-J686 
 
 
 
 
 BoSSS-J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M466.4,93.1 L519.8,93.1 M80.2,137.5 L80.7,137.5 L81.2,137.6 L81.6,137.7 L82.1,137.8 L82.6,138.0
 L83.1,138.2 L83.5,138.5 L84.0,138.8 L84.5,139.1 L85.0,139.5 L85.5,140.0 L85.9,140.4 L86.4,140.9
 L86.9,141.5 L87.4,142.1 L87.9,142.7 L88.3,143.4 L88.8,144.1 L89.3,144.8 L89.8,145.6 L90.2,146.5
 L90.7,147.3 L91.2,148.2 L91.7,149.1 L92.2,150.1 L92.6,151.1 L93.1,152.2 L93.6,153.2 L94.1,154.4
 L94.6,155.5 L95.0,156.7 L95.5,157.9 L96.0,159.2 L96.5,160.4 L96.9,161.8 L97.4,163.1 L97.9,164.5
 L98.4,165.9 L98.9,167.4 L99.3,168.8 L99.8,170.4 L100.3,171.9 L100.8,173.5 L101.3,175.1 L101.7,176.7
 L102.2,178.3 L102.7,180.0 L103.2,181.8 L103.6,183.5 L104.1,185.3 L104.6,187.1 L105.1,189.0 L105.6,190.8
 L106.0,192.7 L106.5,194.6 L107.0,196.6 L107.5,198.5 L108.0,200.5 L108.4,202.6 L108.9,204.6 L109.4,206.7
 L109.9,208.8 L110.3,210.9 L110.8,213.0 L111.3,215.2 L111.8,217.5 L112.3,219.7 L112.7,221.9 L113.2,224.2
 L113.7,226.5 L114.2,228.8 L114.7,231.2 L115.1,233.5 L115.6,235.9 L116.1,238.3 L116.6,240.7 L117.0,243.2
 L117.5,245.6 L118.0,248.1 L118.5,250.5 L119.0,253.0 L119.4,255.5 L119.9,258.1 L120.4,260.6 L120.9,263.2
 L121.4,265.8 L121.8,268.4 L122.3,271.0 L122.8,273.6 L123.3,276.3 L123.7,278.9 L124.2,281.5 L124.7,284.2
 L125.2,286.9 L125.7,289.5 L126.1,292.2 L126.6,294.9 L127.1,297.6 L127.6,300.3 L128.1,303.0 L128.5,305.8
 L129.0,308.5 L129.5,311.2 L130.0,314.0 L130.4,316.7 L130.9,319.4 L131.4,322.2 L131.9,324.9 L132.4,327.7
 L132.8,330.4 L133.3,333.1 L133.8,335.9 L134.3,338.6 L134.7,341.3 L135.2,344.1 L135.7,346.8 L136.2,349.5
 L136.7,352.2 L137.1,354.9 L137.6,357.6 L138.1,360.3 L138.6,363.0 L139.1,365.7 L139.5,368.3 L140.0,371.0
 L140.5,373.6 L141.0,376.3 L141.4,378.9 L141.9,381.5 L142.4,384.1 L142.9,386.7 L143.4,389.3 L143.8,391.8
 L144.3,394.4 L144.8,396.9 L145.3,399.4 L145.8,401.9 L146.2,404.4 L146.7,406.9 L147.2,409.3 L147.7,411.8
 L148.1,414.2 L148.6,416.6 L149.1,419.0 L149.6,421.3 L150.1,423.7 L150.5,426.0 L151.0,428.4 L151.5,430.7
 L152.0,432.9 L152.5,435.1 L152.9,437.3 L153.4,439.5 L153.9,441.6 L154.4,443.8 L154.8,445.9 L155.3,448.0
 L155.8,450.1 L156.3,452.1 L156.8,454.1 L157.2,456.2 L157.7,458.2 L158.2,460.1 L158.7,462.0 L159.2,463.9
 L159.6,465.8 L160.1,467.7 L160.6,469.5 L161.1,471.4 L161.5,473.1 L162.0,474.9 L162.5,476.6 L163.0,478.3
 L163.5,480.0 L163.9,481.6 L164.4,483.2 L164.9,484.8 L165.4,486.4 L165.9,487.9 L166.3,489.4 L166.8,490.8
 L167.3,492.3 L167.8,493.7 L168.2,495.1 L168.7,496.4 L169.2,497.7 L169.7,499.0 L170.2,500.3 L170.6,501.5
 L171.1,502.7 L171.6,503.9 L172.1,505.1 L172.6,506.2 L173.0,507.3 L173.5,508.3 L174.0,509.3 L174.5,510.3
 L174.9,511.3 L175.4,512.3 L175.9,513.2 L176.4,514.0 L176.9,514.9 L177.3,515.7 L177.8,516.5 L178.3,517.3
 L178.8,518.0 L179.2,518.7 L179.7,519.4 L180.2,520.0 L180.7,520.6 L181.2,521.2 L181.6,521.8 L182.1,522.1
 L182.6,522.7 L183.1,523.2 L183.6,523.6 L184.0,524.1 L184.5,524.5 L185.0,524.8 L185.5,525.2 L185.9,525.5
 L186.4,525.7 L186.9,526.0 L187.4,526.2 L187.9,526.4 L188.3,526.5 L188.8,526.6 L189.3,526.7 L189.8,526.8
 L190.3,526.8 L190.7,526.8 L191.2,526.8 L191.7,526.8 L192.2,526.7 L192.6,526.6 L193.1,526.5 L193.6,526.3
 L194.1,526.1 L194.6,525.9 L195.0,525.7 L195.5,525.4 L196.0,525.2 L196.5,524.8 L197.0,524.5 L197.4,524.1
 L197.9,523.7 L198.4,5